In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()

22/06/14 10:40:49 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:40:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:40:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:40:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:40:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:40:51 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:40:51 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

### Reading JSON into DataFrame

In [5]:
df = spark.read.json('../data/people.json')

In [6]:
# show data
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
# view schema
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
# view columns
df.columns

['age', 'name']

In [9]:
# get numeric statistics with describe
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



### Explicitly Defining Schemas
- sometimes we need to clarify the schema of dataframe
- Spark lets us do this with pyspark.sql.types package

In [12]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

# name, datatype, is nullable
data_schema = [
    StructField('age', IntegerType(), True),
    StructField('name', StringType(), True)
]

final_struct = StructType(fields=data_schema)
df = spark.read.json('../data/people.json', schema=final_struct)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Selecting Data from DataFrame

In [13]:
# indexing the data frame returns a column descriptor, not the data
type(df['age'])

pyspark.sql.column.Column

In [14]:
# select will return a DataFrame
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [15]:
# head returns a list of Rows
type(df.head(2)[0])

pyspark.sql.types.Row

In [16]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating New Columns

In [17]:
# return new DataFrame by adding/replacing a column
df.withColumn('2x age', df['age']*2).show()

+----+-------+------+
| age|   name|2x age|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    60|
|  19| Justin|    38|
+----+-------+------+



In [18]:
df.withColumn('age + 5', df['age'] + 5).show()

+----+-------+-------+
| age|   name|age + 5|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     35|
|  19| Justin|     24|
+----+-------+-------+



In [19]:
# rename a column
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



### Using SQL to Interact with DataFrame
- you can leverage your SQL knowledge with Spark.sql

In [20]:
# register DataFrame as SQL Temp View
df.createOrReplaceTempView('people')

In [21]:
results = spark.sql("SELECT name as NAME from people WHERE age = 30")

In [22]:
results.show()

+----+
|NAME|
+----+
|Andy|
+----+



In [23]:
new_results = spark.sql